In [ ]:
import newBlock as nb
import cartopy
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import metpy.calc as mpcalc
import json
import calendar
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 

import matplotlib.ticker as mticker
import matplotlib.font_manager as font_manager
import matplotlib.patches as mpatches
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import num2date
import matplotlib.pyplot as plt

In [ ]:
dataset = nb.dataset('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/msl_79_20.nc', 'ERA-5')
dados = (dataset.formatar(lat=[-20, -40], lon=[-42, 12], time=["1979-12-01", "2019-11-01"], opr="divisao", fator=100)).dataset

In [ ]:
def season_coords(ds, periodo):
    
    # Nesta função o dataset(ds) original é recortado para o período de análise e retorna uma LISTA DE DATASETS e UM DATASET
    # A lista 'data_list' retornada contém os datasets.season de cada intervalo de UM ANO dentro do RANGE do 'periodo de recorte'
    # EX:: ASSUMINDO QUE 'PERIODO' É [1979-12-01, 2010-11-01] <--> 'DATA_LIST[0]' retornará o dataset.season do intervalo de ..
    # ... (Continuando)... >> [1979-12-01 até 1980-11-01] <<< EQUIVALENTE AO INTERVALO UM ANO (DATA_LIST[0])
    # O dataset 'data_season' retornado é o dataset.season de TODO o RANGE do 'periodo de recorte'
    # Logo, o 'data_season' é a média.SEASON do intervalo completo do dataset recortado
    # EX:: ASSUMINDO QUE PERIODO >> [1979-12-01, 2010-11-01] << --> 'DATA_SEASON' retornará o dataset.season do PERIODO
    
    data = ds.sel(time=slice(periodo[0], periodo[1]))
    data_season = data.groupby("time.season").mean()
    data_list = []
    for i in range(0, len(data.time.values), 12):
        d_slice = data.isel(time=slice(i, i+12))
        d_season = d_slice.groupby("time.season").mean()
        data_list.append(d_season)
        
    return data_list, data_season

In [ ]:
def season_scat(ax, ds, season, hems, periodo, periodobase=None, media_periodo=False, media_base=False, only_media=False):
    
    # Nesta função é plotado o gráfico de distribuição 'SCATTER_PLOT' dos pontos máximos de pressão gerados 
    # Função suporta até DOIS períodos para comparação --> periodobase = periodo opcional para a comparação (climatologias)
    # Os parâmetros 'periodo' e 'periodobase' plotam os pontos de máxima para os datasets.season (DATA_LIST) ...
    #... (Continuando)... retornados para seus respectivos intervalos de 'periodo' e 'periodobase', enquanto os ...
    #... (Continuando)... *KWARGS 'media_periodo' e 'media_base' plotam para o dataset.season (DATA_SEASON) from 'SEASON_COORDS' 
    # O parametro only_media é um *KWARG que permite ao usuário, caso TODOS os outros *kwargs sejam =! dos valores default ...
    #... (Continuando)..., ou seja << periodobase != NONE AND medias !=FALSE >>, plotar somente os valores (DATA_SEASON)
    
    
    # IMPORTANDO 'DATA_SEASON' E 'DATA_LIST' CORRESPONDENTES AO INTERVALOR 'PERIODO'
    seasons, seasons_mean = season_coords(ds, periodo)
    
    # LISTA COM AS LABELS DOS SCATTER POINTS
    handles = []
    
    # CUIDADO: --> 'summer' é o verão do hemisfério Norte (JJA) <-> inverno do hemisfério (SUL) 'inverno'
    if hems == "Sul":
        if season == "verão" or season =="ver":
            season_index = 0
        elif season == "inverno" or season == "inv":
            season_index = 1
        elif season == "outono" or season == "out":
            season_index = 2
        elif season == "primavera" or season == "pri":
            season_index = 3
            
    elif hems == "Norte":
        if season == "winter" or season == "win":
            season_index = 0
        elif season == "summer" or season == "sum":
            season_index = 1
        elif season == "spring" or season == "spr":
            season_index = 2
        elif season == "autumn" or season == "aut":
            season_index = 3
        
    if only_media == False: 
        
        # Média Sazonal para cada intervalos de um ano <-> MÚLTIPLOS scatter points (!!!)
        # Formatando o dataset para o recorte do valor máximo de 'msl', retornando seu VALOR e COORDS
        # Plotando os scatter points <-> pontos de máximo
        for data in seasons:
            max_mensal = data.isel(season=season_index)
            max_data = max_mensal.where(max_mensal==max_mensal.msl.max(), drop=True)
            latitude = np.nanmax(max_data.lat)
            longitude = np.mean(max_data.lon)
            valores, xs, ys = max_data.msl, longitude, latitude         
            ax.scatter(xs, ys, transform=ccrs.PlateCarree(), color='orangered', alpha=0.5)
        cyan_patch_1 = mpatches.Patch(color='orangered', label=f"{periodo[0][0:4]} to {periodo[1][0:4]}", capstyle="round")
        handles.append(cyan_patch_1)
        
    if media_periodo == True:
        
        # Média sazonal para o período completo <-> ÚNICO scatter point (!!!)
        # Formatando o dataset para o recorte do valor máximo de 'msl', retornando seu VALOR e COORDS
        # Plotando os scatter points <-> pontos de máximo
        max_season_mean = seasons_mean.isel(season=season_index)
        max_data_mean = max_season_mean.where(max_season_mean==max_season_mean.msl.max(), drop=True)
        latitude_mean = np.nanmax(max_data_mean.lat)
        longitude_mean = np.mean(max_data_mean.lon)
        valores, xs, ys = max_data_mean.msl, longitude_mean, latitude_mean  
        if only_media == False:
            ax.scatter(xs, ys, s=300, transform=ccrs.PlateCarree(), color='orangered', alpha=1, marker='v')
        else:
            ax.scatter(xs, ys, s=200, transform=ccrs.PlateCarree(), color='orangered', alpha=1, marker='x', label=f"{periodo[0][0:4]} to {periodo[1][0:4]}", capstyle="round")

    if periodobase != None:
        # IMPORTANDO 'DATA_SEASON' E 'DATA_LIST' CORRESPONDENTES AO INTERVALOR 'PERIODOBASE'
        seasons_2, seasons_mean_2 = season_coords(ds, periodobase)
        
        if only_media == False:
            # MESMOS RACIOCÍNIOS, AGORA APLICADOS PARA O SEGUNDO PERIODO DE COMPARAÇÃO (periodobase) != None
#            for data in seasons_2:
#                max_mensal_2 = data.isel(season=season_index)
#                max_data_2 = max_mensal_2.where(max_mensal_2==max_mensal_2.msl.max(), drop=True)
#                latitude_2 = np.nanmax(max_data_2.lat)
#                longitude_2 = np.mean(max_data_2.lon)
#                valores, xs, ys = max_data_2.msl, longitude_2, latitude_2         
#                ax.scatter(xs, ys, transform=ccrs.PlateCarree(), color='darkcyan', alpha=0.5)
            cyan_patch_3 = mpatches.Patch(color='darkcyan', label=f"{periodobase[0][0:4]} to {periodobase[1][0:4]}", capstyle="round")
            handles.append(cyan_patch_3)
            
        if media_base == True:
            # MESMOS RACIOCÍNIOS, AGORA APLICADOS PARA O SEGUNDO PERIODO DE COMPARAÇÃO (periodobase) != None
            max_season_mean_2 = seasons_mean_2.isel(season=season_index)
            max_data_mean_2 = max_season_mean_2.where(max_season_mean_2==max_season_mean_2.msl.max(), drop=True)
            latitude_mean_2 = np.nanmax(max_data_mean_2.lat)
            longitude_mean_2 = np.mean(max_data_mean_2.lon)
            valores, xs, ys = max_data_mean_2.msl, longitude_mean_2, latitude_mean_2
            if only_media == False:
                ax.scatter(xs, ys, s=300, transform=ccrs.PlateCarree(), color='darkcyan', alpha=1, marker='v')
            else:
                ax.scatter(xs, ys, s=200, transform=ccrs.PlateCarree(), color='darkcyan', alpha=1, marker="x", label=f"{periodobase[0][0:4]} to {periodobase[1][0:4]}", capstyle="round")
   
    if only_media == True:
        # TÍTULO DA LEGENDA QUANDO HÁ SÓ OS VALORES DE 'DATA_SEASON' <-> 'ONLY_MEDIA == TRUE'
        ax.legend(title="Max Position", title_fontsize=12, loc="upper right", fancybox=True, shadow=True, labelspacing=1.0, borderpad=0.7)
    else:
        # SEM TÍTULO DE LEGENDA PARA 'ONLY_MEDIA == FALSE'
        ax.legend(handles=handles, prop=font_manager.FontProperties(style='oblique', size=12), loc="upper right", fancybox=True, shadow=True)
        
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':', edgecolor='gray')

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-180, 200, 10))
    gl.ylocator = mticker.FixedLocator(np.arange(-180, 200, 5))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}

    ax.set_extent([-42, 12, -40, -20], crs=ccrs.PlateCarree())         
    ax.coastlines()
    ax.set_title(seasons_mean.season[season_index].values, loc='left', fontsize=20, style="oblique")
    ax.set_title("LAMMOC-UFF", loc='right', fontsize=9, style="oblique")
    

In [ ]:
def figura(hems, periodo, periodobase=None, media_periodo=False, media_base=False, only_media=False):
    
    fig, ax = plt.subplots(ncols = 2, nrows = 2, subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(20, 10))
    fig.suptitle("Maximum Sea Level Pressure by Season", fontsize=30, fontstyle='oblique')
    
    if hems == "Sul":
        season_scat(ax[0,0], dados, "ver", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[0,1], dados, "inv", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[1,0], dados, "out", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[1,1], dados, "pri", hems, periodo, periodobase, media_periodo, media_base, only_media)
        
    if hems == "Norte":
        season_scat(ax[0,0], dados, "win", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[0,1], dados, "sum", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[1,0], dados, "spr", hems, periodo, periodobase, media_periodo, media_base, only_media)
        season_scat(ax[1,1], dados, "aut", hems, periodo, periodobase, media_periodo, media_base, only_media)

    # plt.savefig("MSL_MAX_Scatt_Mean.png")
    # plt.savefig("MSL_MAX_Mean.png")
    # plt.savefig("MSL_MAX_Scatt.png")


In [ ]:
figura("Sul", ["2010-12-01", "2018-11-01"], ["1979-12-01", "2010-11-01"], media_periodo=True, media_base=True, only_media=True)


In [ ]:
figura("Norte", ["2017-12-01", "2020-11-01"], ["1979-12-01", "2010-11-01"], media_periodo=False, media_base=True)
plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Apresentações/MODCLIM/17_20Centers_mediaClimat')